In [ ]:
# Import additional libraries needed for analysis
import pandas as pd
import numpy as np
import wrds
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns



# Data Cleaning and Feature Engineering
# Replace 'your_username' with your actual WRDS username
wrds_username = 'your_username'

# Connect to WRDS
db = wrds.Connection(wrds_username=wrds_username)

# Query for financial statement data
# Adjust the date range and variables as needed
financial_data_query = """
SELECT gvkey, datadate, at, invt, sale, rect, act, lct, dlc, txp, dp, oiadp, xint, txt
FROM comp.funda
WHERE datadate >= '2010-01-01' AND datadate <= '2020-12-31'
AND indfmt='INDL'
AND datafmt='STD'
AND popsrc='D'
AND consol='C'
"""

financial_data = db.raw_sql(financial_data_query)

# Query for stock returns data
# Adjust the date range as needed
returns_data_query = """
SELECT permno, date, ret
FROM crsp.msf
WHERE date >= '2010-01-01' AND date <= '2020-12-31'
"""

returns_data = db.raw_sql(returns_data_query)

# Close the WRDS database connection
db.close()
# Convert dates and ensure correct data types
merged_data['datadate'] = pd.to_datetime(merged_data['datadate'])
merged_data['date'] = pd.to_datetime(merged_data['date'])
merged_data['ret'] = pd.to_numeric(merged_data['ret'], errors='coerce')

# Calculate year-over-year inventory change as a percentage of total assets
merged_data['invt_change'] = merged_data.groupby('gvkey')['invt'].pct_change() * 100

# More sophisticated accruals calculation
merged_data['accruals'] = ((merged_data['act'] - merged_data['lct']) -
                           (merged_data['dlc'] - merged_data['txp']) -
                           merged_data['dp']) / merged_data['at'].shift(1)

# Additional financial ratios
merged_data['profit_margin'] = merged_data['oiadp'] / merged_data['sale']
merged_data['asset_turnover'] = merged_data['sale'] / merged_data['at'].shift(1)
merged_data['current_ratio'] = merged_data['act'] / merged_data['lct']
merged_data['quick_ratio'] = (merged_data['act'] - merged_data['invt']) / merged_data['lct']
merged_data['debt_to_equity'] = (merged_data['dlc'] + merged_data['txp']) / merged_data['at']

# Lag the return data by one period for future returns analysis
merged_data['future_ret'] = merged_data.groupby('gvkey')['ret'].shift(-1)

# Drop rows with missing data
merged_data.dropna(inplace=True)

# Exploratory Data Analysis (EDA)

# Visualizations for understanding data distribution
plt.figure(figsize=(12, 8))
sns.histplot(merged_data['invt_change'], kde=True)
plt.title('Inventory Change Distribution')
plt.xlabel('Inventory Change (%)')
plt.ylabel('Frequency')
plt.show()

# Correlation matrix heatmap
plt.figure(figsize=(12, 8))
sns.heatmap(merged_data[['invt_change', 'accruals', 'profit_margin', 'asset_turnover', 'future_ret']].corr(), annot=True, cmap='coolwarm')
plt.title('Correlation Matrix')
plt.show()

# Advanced Statistical Analysis

# Regression Analysis with Multiple Variables
y = merged_data['future_ret']
X = merged_data[['invt_change', 'accruals', 'profit_margin', 'asset_turnover', 'current_ratio', 'quick_ratio', 'debt_to_equity']]
X = sm.add_constant(X)

model = sm.OLS(y, X)
results = model.fit()
print(results.summary())

# Normality Test for Residuals
jb_test = stats.jarque_bera(results.resid)
print("\nJarque-Bera test for normality of residuals:")
print("JB Statistic:", jb_test[0])
print("p-value:", jb_test[1])

# Test for Heteroskedasticity
bp_test = sm.stats.diagnostic.het_breuschpagan(results.resid, results.model.exog)
print("\nBreusch-Pagan test for heteroskedasticity:")
print("BP Statistic:", bp_test[0])
print("p-value:", bp_test[1])

# Multicollinearity Check
vif_data = pd.DataFrame()
vif_data["feature"] = X.columns
vif_data["VIF"] = [sm.stats.outliers_influence.variance_inflation_factor(X.values, i) for i in range(len(X.columns))]
print("\nVariance Inflation Factor (VIF) for multicollinearity check:")
print(vif_data)

# Time-Series Analysis
# Implementing a time-series analysis (e.g., ARIMA, GARCH) based on stock returns

# Further Analysis
# Additional analyses like sector-based analysis, firm size effect, and economic cycle impact can be included

# Note: Each section above can be further expanded with more detailed analyses and customizations
# Import additional libraries needed for analysis
import pandas as pd
import numpy as np
import wrds
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns

# Connect to WRDS and retrieve data (not shown here, refer to the previous script)

# Data Cleaning and Feature Engineering

# Convert dates and ensure correct data types
merged_data['datadate'] = pd.to_datetime(merged_data['datadate'])
merged_data['date'] = pd.to_datetime(merged_data['date'])
merged_data['ret'] = pd.to_numeric(merged_data['ret'], errors='coerce')

# Calculate year-over-year inventory change as a percentage of total assets
merged_data['invt_change'] = merged_data.groupby('gvkey')['invt'].pct_change() * 100

# More sophisticated accruals calculation
merged_data['accruals'] = ((merged_data['act'] - merged_data['lct']) -
                           (merged_data['dlc'] - merged_data['txp']) -
                           merged_data['dp']) / merged_data['at'].shift(1)

# Additional financial ratios
merged_data['profit_margin'] = merged_data['oiadp'] / merged_data['sale']
merged_data['asset_turnover'] = merged_data['sale'] / merged_data['at'].shift(1)
merged_data['current_ratio'] = merged_data['act'] / merged_data['lct']
merged_data['quick_ratio'] = (merged_data['act'] - merged_data['invt']) / merged_data['lct']
merged_data['debt_to_equity'] = (merged_data['dlc'] + merged_data['txp']) / merged_data['at']

# Lag the return data by one period for future returns analysis
merged_data['future_ret'] = merged_data.groupby('gvkey')['ret'].shift(-1)

# Drop rows with missing data
merged_data.dropna(inplace=True)

# Exploratory Data Analysis (EDA)

# Visualizations for understanding data distribution
plt.figure(figsize=(12, 8))
sns.histplot(merged_data['invt_change'], kde=True)
plt.title('Inventory Change Distribution')
plt.xlabel('Inventory Change (%)')
plt.ylabel('Frequency')
plt.show()

# Correlation matrix heatmap
plt.figure(figsize=(12, 8))
sns.heatmap(merged_data[['invt_change', 'accruals', 'profit_margin', 'asset_turnover', 'future_ret']].corr(), annot=True, cmap='coolwarm')
plt.title('Correlation Matrix')
plt.show()

# Advanced Statistical Analysis

# Regression Analysis with Multiple Variables
y = merged_data['future_ret']
X = merged_data[['invt_change', 'accruals', 'profit_margin', 'asset_turnover', 'current_ratio', 'quick_ratio', 'debt_to_equity']]
X = sm.add_constant(X)

model = sm.OLS(y, X)
results = model.fit()
print(results.summary())

# Normality Test for Residuals
jb_test = stats.jarque_bera(results.resid)
print("\nJarque-Bera test for normality of residuals:")
print("JB Statistic:", jb_test[0])
print("p-value:", jb_test[1])

# Test for Heteroskedasticity
bp_test = sm.stats.diagnostic.het_breuschpagan(results.resid, results.model.exog)
print("\nBreusch-Pagan test for heteroskedasticity:")
print("BP Statistic:", bp_test[0])
print("p-value:", bp_test[1])

# Multicollinearity Check
vif_data = pd.DataFrame()
vif_data["feature"] = X.columns
vif_data["VIF"] = [sm.stats.outliers_influence.variance_inflation_factor(X.values, i) for i in range(len(X.columns))]
print("\nVariance Inflation Factor (VIF) for multicollinearity check:")
print(vif_data)

